## Parsing unity VR log file

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os.path import sep
import json

from dataclasses import dataclass

In [2]:
from unityvr.preproc import logproc
from unityvr.viz import viz

In [3]:
dirName = "../sample/test2"
fileName = "Log_2020-8-29_20-53-53_SS96_7f_f01_1.json"

dat = logproc.openUnityLog(dirName, fileName)

for i in range(1):
    print(dat[i]['frame'])
    print(json.dumps(dat[i]['data'], sort_keys=True, indent=4))

1.0
{
    "colliderType": "MeshCollider",
    "meshGameObjectPath": "Fly/FlyCamera3/FlyCamera3Screen",
    "worldPosition": {
        "x": 38.428314208984375,
        "y": 30.339401245117188,
        "z": -52.89203643798828
    },
    "worldRotationDegs": {
        "x": 0.0,
        "y": 324.0,
        "z": 0.0
    },
    "worldScale": {
        "x": 95.00000762939453,
        "y": 182.0,
        "z": 1.0
    }
}


In [4]:
#def loadUnityLog(dirName, fileName):

#def parseUnityLog(dat):

### Construct data object from log file
#### Pull out object positions as pandas dataframe

In [5]:
objDf = logproc.objDfFromLog(dat)
objDf

,name,collider,px,py,pz,rx,ry,rz,sx,sy,sz
0,Fly/FlyCamera3/FlyCamera3Screen,MeshCollider,38.428314,-52.892036,30.339401,0.0,0.0,324.000000,95.000008,1.0,182.0
1,Fly/FlyCamera1/FlyCamera1Screen,MeshCollider,0.000000,65.378143,30.339401,0.0,0.0,180.000000,95.000000,1.0,182.0
2,Fly/FlyCamera2/FlyCamera2Screen,MeshCollider,62.178310,20.202957,30.339401,0.0,0.0,252.000000,95.000008,1.0,182.0
3,LMlocal1,CapsuleCollider,0.000000,-100.000000,27.000000,0.0,0.0,0.000000,10.000000,10.0,30.0
4,LMlocal2,CapsuleCollider,-150.000000,100.000000,27.000000,0.0,0.0,0.000000,10.000000,10.0,30.0
5,Fly/FlyCamera4/FlyCamera4Screen,MeshCollider,-38.428303,-52.892036,30.339401,0.0,0.0,35.999996,95.000000,1.0,182.0


#### Pull out fly position as pandas dataframe
Position in VR

In [ ]:
posDf = logproc.posDfFromLog(dat)
posDf.head()

Position based on Fictrac tracking

In [ ]:
ftDf = logproc.ftDfFromLog(dat)
ftDf.head()

Fictrac updates come in at a higher rate than the VR frame rate. Fictrac can also only run for a subset of the VR run time and therefore the number of frames in fictrac and vr dataframes may not match.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))
ax.plot(posDf.frame, 'k')
ax.plot(ftDf.frame, 'r')
ax.legend(['vr position update frames','fictrac update frames'])

#### Generate data object

In [ ]:
# to be moved to logproc

objDfCols = ['name','collider','px','py','pz','rx','ry','rz','sx','sy','sz']

posDfCols = ['frame','time','x','y','angle']
ftDfCols = ['frame','ficTracTReadMs','ficTracTWriteMs','dx','dy','dz']

@dataclass
class unityVRexperiment:

    # metadata
    experiment: str
    genotype: str
    sex: str
    flyid: int
    date: str
        
    imaging: bool = False
    brainregion: str = None
    
    # timeseries data
    posDf: pd.DataFrame = pd.DataFrame(columns=posDfCols)
    ftDf: pd.DataFrame = pd.DataFrame(columns=ftDfCols)
        
    # object locations
    objDf: pd.DataFrame = pd.DataFrame(columns=objDfCols)
    
    # methods
    def printMetadata(self):
        print("Experiment {} with {} (fly {}{})\nDate: {}\nImaging: {}".format(self.experiment,
                                    self.genotype, self.sex,self.flyid, self.date,self.imaging))
    
    ## data wrangling
    def downsampleftDf(self):
        frameftDf = ftDf.groupby("frame").sum()
        frameftDf.reset_index(level=0, inplace=True)
        return frameftDf
    
    

In [ ]:
uvrTest = unityVRexperiment('test','testgenotype','f',1,'testdate',posDf=posDf,ftDf=ftDf,objDf=objDf)
uvrTest.printMetadata()

In [ ]:
frameftDf = uvrTest.downsampleftDf()
frameftDf.head()

### Vizualize

#### Frame rate

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(10,10))

axs[0].plot(uvrTest.posDf.frame, np.gradient(uvrTest.posDf.time))
axs[1].plot(uvrTest.ftDf.frame, np.gradient(uvrTest.ftDf.ficTracTReadMs/1000.))
axs[2].plot(uvrTest.downsampleftDf().frame, uvrTest.downsampleftDf().ficTracTReadMs/1000.)

#### Vizualize trajectory and object positions

In [ ]:
ballr = 45
#plt.plot(posDf['x'], posDf['y'],color='grey',alpha=0.5)
#plt.scatter(posDf['x'], posDf['y'],s=7,c=posDf['time'],cmap='viridis')

# See Seelig 2010 for reference on equations (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2945246/)
fta = np.cumsum(ftDf['dz'])
fty = np.cumsum(ballr*np.pi/180*ftDf['dy']*np.cos(fta) - ballr*np.pi/180*ftDf['dx']*np.sin(fta))
ftx = np.cumsum(ballr*np.pi/180*ftDf['dy']*np.sin(fta) + ballr*np.pi/180*ftDf['dx']*np.cos(fta))
plt.plot(ftx[0:3000], fty[0:3000],color='grey',alpha=0.5)
plt.scatter(ftx[0:3000], fty[0:3000],s=7,c=ftDf['frame'][0:3000],cmap='viridis')

In [ ]:
plt.plot(posDf['x'], posDf['y'],color='grey',alpha=0.5)
plt.scatter(posDf['x'], posDf['y'],s=7,c=posDf['angle'],cmap='hsv')

In [ ]:
plt.plot(posDf['x'], posDf['y'],color='grey',alpha=0.5)
plt.scatter(posDf['x'], posDf['y'],s=7,c=posDf['time'],cmap='viridis')

In [ ]:
fig = viz.plotVRpathWithObjects(uvrTest, limx=[-200,200], limy=[-200,200],myfigsize=(10,10))